In [1]:
#First part, do not run again

import pandas as pd

# List of file names
files = ['Day-ahead Prices_2018.csv', 'Day-ahead Prices_2019.csv', 'Day-ahead Prices_2020.csv', 
         'Day-ahead Prices_2021.csv', 'Day-ahead Prices_2022.csv', 'Day-ahead Prices_2023.csv']

#  single DataFrame
dfs = []
for file in files:
    df = pd.read_csv(file)
    dfs.append(df)

df_merged = pd.concat(dfs)

# Drop the 'Currency' and 'BZN' columns
df_merged.drop(['Currency', 'BZN|IE(SEM)'], axis=1, inplace=True)

# Rename the column
df_merged.rename(columns={'MTU (CET/CEST)': 'Time',
                          'Day-ahead Price [EUR/MWh]': 'Day-ahead Price [EUR/MWh]'}, inplace=True)

# Save the merged DataFrame to a new CSV file
df_merged.to_csv('Day-ahead Prices_2018-2023.csv', index=False)



In [2]:
import pandas as pd
# Read the dataset
df_prices = pd.read_csv('Day-ahead Prices_2018-2023.csv')
# Print the column names to verify the exact column name
print(df_prices.columns)

Index(['Time', 'Day-ahead Price [EUR/MWh]'], dtype='object')


In [8]:
#Drop missing values, set 11AM price everyday, resample file

import pandas as pd

# Read the dataset
df_prices = pd.read_csv('Day-ahead Prices_2018-2023.csv')

# Preprocess the 'Time' column
df_prices[['Start Time', 'End Time']] = df_prices['Time'].str.split(' - ', expand=True)
df_prices['Time'] = pd.to_datetime(df_prices['Start Time'], format='%d.%m.%Y %H:%M')
df_prices = df_prices.set_index('Time')

# Define the date ranges to keep
keep_ranges = [(pd.Timestamp('2018-10-01 01:00'), pd.Timestamp('2023-04-12 00:00')),
               (pd.Timestamp('2023-04-26 02:00'), pd.Timestamp('2023-05-31 00:00'))]

# Keep the rows within the specified date ranges only
df_prices = pd.concat([df_prices.loc[start:end] for start, end in keep_ranges])

# Resample the data to daily frequency and forward-fill any missing values
df_prices_daily = df_prices.resample('24H', base=11).first().ffill()

# Select the 'Day-ahead Price [EUR/MWh]' column
selected_column = 'Day-ahead Price [EUR/MWh]'
if selected_column in df_prices_daily.columns:
    df_prices_daily = df_prices_daily[[selected_column]]
    df_prices_daily.to_csv('Day-ahead Prices_Resampled.csv')
else:
    print(f"Column '{selected_column}' not found in the DataFrame.")


C:\Users\ridva\AppData\Local\Temp\ipykernel_15108\259880021.py:19: FutureWarning: 'base' in .resample() and in Grouper() is deprecated.
The new arguments that you should use are 'offset' or 'origin'.

>>> df.resample(freq="3s", base=2)

becomes:

>>> df.resample(freq="3s", offset="2s")

  df_prices_daily = df_prices.resample('24H', base=11).first().ffill()
